In [1]:
!pip install streamlit
!pip install pyngrok==4.1.1
!npm install -g localtunnel


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 113.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15964 sha256=d9f51dda65531f2b90ffd8dc7ad68e4ffb8f5f1a0a56760e48dd87d3ada33da4
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok


/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 3.409s


In [30]:

%%writefile app.py
# Streamlit app code

import pandas as pd
import networkx as nx
from itertools import combinations
import numpy as np
import random
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
import streamlit as st

# Create a graph
def graph(user_data, recipe_data):
    G_new = nx.Graph()


    # adding user nodes
    for user_id in user_data['user_id'].unique():
        G_new.add_node(f"user_{user_id}", node_type='user')

    # Adding recipe nodes along with their attributes
    for _, row in recipe_data.iterrows():
        recipe_id = f"recipe_{row['recipe_id']}"
        recipe_attributes = {
            'contributor_id': row['contributor_id'],
            'calories': row['calories'],
            'recipe_complexity': row['recipe_complexity'],
            'ingredient_ids': list(map(int, row['ingredient_ids'].strip('[]').split(', ')))  # Converting to list of integers
        }
        G_new.add_node(recipe_id, **recipe_attributes, node_type='recipe')

    for _, row in user_data.iterrows():
        user_node = f"user_{row['user_id']}"
        recipe_node = f"recipe_{row['recipe_id']}"
        # Add edge only if both nodes exist
        if user_node in G_new.nodes and recipe_node in G_new.nodes:
            G_new.add_edge(user_node, recipe_node, rating=row['rating'])

    return G_new

def recommend_recipes_full(user_id,user, G, model,scaler,user_data,recipe_data,ingredients_list,ingr_map, top_n=100):
    user_mean = user_data[user_data['user_id'] == user]['rating'].mean()
    # Find recipes rated by the user
    user_rated_recipes = [node for node, attr in G[user_id].items() if attr['rating'] >= user_mean]  # considering high ratings

    # Group the attributes of liked recipes
    attributes_of_liked_recipes = []
    for recipe in user_rated_recipes:
        attributes_of_liked_recipes.append(G.nodes[recipe])
    sel_ingr = ingr_map[ingr_map['Ingredient'].isin(ingredients_list)]
    # Comparing these attributes with other recipes
    recommendations = []
    for node, attr in G.nodes(data=True):
        if attr['node_type'] == 'recipe' and node not in user_rated_recipes:
            similarity_score = 0  # A metric to measure similarity with liked recipes

            for liked_recipe_attr in attributes_of_liked_recipes:
                if (len(set(sel_ingr['ID']).intersection(set(attr['ingredient_ids']))) != len(set(sel_ingr['ID']))):
                  continue
                else:
                  similarity_score += len(set(sel_ingr['ID']))
                # Ingredient similarity
                shared_ingredients = len(set(liked_recipe_attr['ingredient_ids']).intersection(set(attr['ingredient_ids'])))
                similarity_score += shared_ingredients

                # recipe complexity similarity
                complexity_difference = abs(liked_recipe_attr['recipe_complexity'] - attr['recipe_complexity'])
                similarity_score += max(0, 1 - complexity_difference)  # Adding to the score if complexities are close

                # recipe calories similarity
                calorie_difference = abs(liked_recipe_attr['calories'] - attr['calories'])
                similarity_score += max(0, 1 - calorie_difference)  # Adding to the score if complexities are close

                # Considering contributor similarity (if the same contributor, increase similarity)
                if liked_recipe_attr['contributor_id'] == attr['contributor_id']:
                    similarity_score += 1

            recommendations.append((node, similarity_score))

    # Sort the recommendations based on similarity score and return the top N recommendatons
    sorted_recommendations = sorted(recommendations, key=lambda x: x[1], reverse=True)
    full_recommendations = [recipe for recipe, _ in sorted_recommendations[:top_n]]
    for kk in range(len(full_recommendations)):
        full_recommendations[kk] = full_recommendations[kk].replace('recipe_', '')
        full_recommendations[kk] = int(full_recommendations[kk])


    unrated_recipes_df = recipe_data[recipe_data['recipe_id'].isin(full_recommendations)]
    unrated_recipes_df[['calories_norm', 'recipe_complexity_norm']] = scaler.fit_transform(unrated_recipes_df[['calories', 'recipe_complexity']])

    user_id_to_recommend = user

    user_data_to_predict = np.array([user_id_to_recommend] * len(unrated_recipes_df))
    recipe_data_to_predict = unrated_recipes_df['recipe_id'].values
    features_data_to_predict = unrated_recipes_df[['calories_norm', 'recipe_complexity_norm']].values

    # Predict the ratings with the model
    predicted_ratings = model.predict([user_data_to_predict, recipe_data_to_predict, features_data_to_predict])

    # Add the predicted ratings to the unrated recipes dataframe
    unrated_recipes_df['predicted_rating'] = predicted_ratings

    # Get the top 10 recommended recipes
    top_10_recipes = unrated_recipes_df.sort_values(by='predicted_rating', ascending=False).head(10)

    return top_10_recipes

def main():
    recipe_data = pd.read_csv('recipe_pp.csv')
    user_data = pd.read_csv('user_pp.csv')
    RAW_recipes = pd.read_csv('RAW_recipes.csv')
    ingr_map = pd.read_csv('ingredient_to_id_mapping_new.csv')
    g_new = graph(user_data, recipe_data)

    st.title('Recipe Recommendation System')
    model = load_model('recipe_recommendation_model3.h5')
    scaler = MinMaxScaler()

    # User input for the user_id
    user_input_id = st.text_input('Enter your user ID:', '')
    ingredients_input = st.text_area('Enter the ingredients you want to use, separated by commas:')

    # Button to make the prediction
    if st.button('Get Recommendations'):
        if user_input_id:
            try:
                user_id = int(user_input_id)
                # Check if the user_id exists in the user dataset
                if user_id in user_data['user_id'].unique():
                    ingredients_list = [ingredient.strip() for ingredient in ingredients_input.split(',') if ingredient.strip()]

                    #Call model prediction function
                    user_id_str = "user_{}".format(user_id)
                    recommendations = recommend_recipes_full(user_id_str,user_id, g_new, model,scaler, user_data,recipe_data, ingredients_list, ingr_map)
                    # Display the top 10 recommendations

                    recipe_selected = RAW_recipes[RAW_recipes['id'].isin(recommendations['recipe_id'].values)]

                    # Dropdown to select a recipe
                    recipe_names = recipe_selected['name'].tolist()
                    selected_recipe = st.selectbox('Select a Recipe', recipe_names)

                    # Display the recipe details
                    if selected_recipe:
                        # Get the row for the selected recipe
                        recipe_info = recipe_selected[recipe_selected['name'] == selected_recipe].iloc[0]

                        st.header(recipe_info['name'])
                        st.subheader('Details')
                        st.write(f"**ID:** {recipe_info['id']}")
                        st.write(f"**Cooking Time:** {recipe_info['minutes']} minutes")
                        st.write(f"**Contributor ID:** {recipe_info['contributor_id']}")
                        st.write(f"**Submitted On:** {recipe_info['submitted']}")

                        st.subheader('Nutrition')
                        st.write(recipe_info['nutrition'])

                        st.subheader('Ingredients')
                        st.write(recipe_info['ingredients'])

                        st.subheader('Steps')
                        st.write(recipe_info['steps'])

                        st.subheader('Description')
                        st.write(recipe_info['description'])



                else:
                    st.error('User ID not found. Please try a different one.')
            except ValueError:
                st.error('Please enter a valid user ID.')
        else:
            st.error('Please enter a user ID.')


if __name__ == '__main__':
    main()

Overwriting app.py


In [31]:
!streamlit run app.py &>/content/logs.txt &


In [32]:




!lt --port 8501



your url is: https://bright-geckos-play.loca.lt
^C
